In [ ]:
import math
import json
from dataclasses import dataclass, asdict, field
from typing import List, Dict, Tuple, Optional

# =================================================================
# 1. DATA STRUCTURES & CONFIGURATION
# =================================================================

@dataclass
class ContractRequest:
    requester_id: str
    owner_id: str
    features: List[str]                 # e.g., ["load_curve", "appliances"]
    granularity_min: int                # temporal resolution (5, 15, 60)
    duration_hours: int                 # reporting window
    purpose: str                        # e.g., "peer_trading", "forecasting"
    proposed_eps: float                 # requester-proposed epsilon

@dataclass
class DecisionResult:
    decision: str                       # "approve" | "reject" | "counter"
    eps_star: Optional[float]
    delta: float
    counter_params: Optional[Dict] = None
    scores: Dict[str, float] = field(default_factory=dict)
    reason_codes: List[str] = field(default_factory=list)

# --- Scoring Weights (System Constants) ---
ALPHA = {
    "location": 1.0,
    "appliances": 0.7,
    "load_curve": 0.4,
    "aggregate": 0.2,
}

P_SCORE = {
    "billing": 1.0,
    "settlement": 0.95,
    "demand_response": 0.7,
    "forecasting": 0.8,
    "peer_trading": 0.6,
    "grid_balancing": 0.85,
    "tariff_design": 0.5,
    "research_analytics": 0.4,
    "profiling": 0.1,
}

# =================================================================
# 2. SCORING PRIMITIVES
# =================================================================

def get_sensitivity_score(features: List[str]) -> float:
    """Calculates S_x: Sum of sensitivity weights for requested features."""
    return sum(ALPHA.get(f, 0.5) for f in features)

def get_reliability_score(requester_id: str, owner_id: str, history: Dict) -> float:
    """Calculates T_ij: Interaction reliability based on historical signals."""
    rec = history.get((requester_id, owner_id))
    if not rec:
        return 0.3  # Default for unknown requesters

    n_success = min(rec.get("success_count", 0) / 20.0, 1.0)
    quality = min(max(rec.get("quality", 0.0), 0.0), 1.0)
    match = min(max(rec.get("purpose_match", 0.0), 0.0), 1.0)

    return (0.4 * n_success) + (0.35 * quality) + (0.25 * match)

def calculate_utility(eps: float, granularity: int, duration: int) -> float:
    """U(eps, G, D): Quantifies the value of the sanitized data."""
    if eps <= 0: return 0.0
    eps_term = 1.0 - math.exp(-eps / 3.0)
    gran_term = min(60.0 / max(granularity, 1), 4.0) / 4.0
    dur_term = min(duration / 24.0, 1.0)
    return max(0.0, min(1.0, 0.55 * eps_term + 0.25 * gran_term + 0.20 * dur_term))

def calculate_risk(eps: float, Sx: float, H_rem: float) -> float:
    """R(eps, Sx, H): Quantifies disclosure risk and budget pressure."""
    budget_pressure = 1.0 - min(H_rem / 10.0, 1.0)
    # Higher Sx and lower H_rem amplify the risk of the chosen eps
    r = (eps / 10.0) * (0.6 + 0.4 * min(Sx / 2.0, 1.0)) * (0.7 + 0.3 * budget_pressure)
    return max(0.0, min(1.0, r))

def get_eps_min_required(Sx: float, Pp: float) -> float:
    """Constraint: Min epsilon required to justify the sensitivity and purpose."""
    base = 0.8
    sens_penalty = 0.6 * min(Sx / 2.0, 1.0)
    purpose_penalty = 0.8 * (1.0 - Pp)
    return base + sens_penalty + purpose_penalty

# =================================================================
# 3. APBNP OPTIMIZATION ENGINE
# =================================================================

def optimize_eps(req: ContractRequest, H_rem: float, Sx: float, Tij: float, Pp: float, lambdas: Dict) -> Dict:
    """Internal loop to find the best eps* for a specific configuration."""
    best_eps = None
    best_obj = -float('inf')

    # Search grid for epsilon [0.25, H_rem]
    eps_limit = min(10.0, H_rem)
    if eps_limit < 0.25:
        return {"eps_star": None, "obj": -1.0, "feasible": False}

    steps = int(eps_limit / 0.25)
    candidates = [round((i + 1) * 0.25, 2) for i in range(steps)]

    eps_min = get_eps_min_required(Sx, Pp)

    for eps in candidates:
        U = calculate_utility(eps, req.granularity_min, req.duration_hours)
        R = calculate_risk(eps, Sx, H_rem)
        C = min(eps / 20.0, 1.0) # Small cost term

        # Multi-objective optimization: Maximize Utility + Reliability + Purpose - Risk - Cost
        obj = (lambdas["u"] * U - lambdas["r"] * R +
               lambdas["t"] * Tij + lambdas["p"] * Pp - lambdas["c"] * C)

        if obj > best_obj:
            best_obj = obj
            best_eps = eps

    feasible = (best_eps is not None) and (best_eps >= eps_min)
    return {"eps_star": best_eps, "obj": best_obj, "feasible": feasible, "eps_min": eps_min}

def apbnp_negotiate(req: ContractRequest, H_rem: float, history: Dict, lambdas: Dict) -> DecisionResult:
    """Main entry point for negotiation and counter-offering."""
    Sx = get_sensitivity_score(req.features)
    Tij = get_reliability_score(req.requester_id, req.owner_id, history)
    Pp = P_SCORE.get(req.purpose, 0.3)

    # 1. Initial Attempt
    res = optimize_eps(req, H_rem, Sx, Tij, Pp, lambdas)

    scores = {
        "Sx": round(Sx, 3), "Tij": round(Tij, 3), "Pp": round(Pp, 3),
        "H_rem": round(H_rem, 3), "eps_min": round(res["eps_min"], 3),
        "objective": round(res["obj"], 3)
    }

    if res["feasible"]:
        scores["U(eps*)"] = round(calculate_utility(res["eps_star"], req.granularity_min, req.duration_hours), 3)
        scores["R(eps*)"] = round(calculate_risk(res["eps_star"], Sx, H_rem), 3)
        return DecisionResult("approve", res["eps_star"], 1e-6, None, scores, ["FEASIBLE"])

    # 2. Counter-Offer Logic (Relax temporal or window constraints)
    reasons = ["INSUFFICIENT_BUDGET_OR_UTILITY"]
    counter_candidates = []
    for g in [15, 30, 60]:
        if g > req.granularity_min: counter_candidates.append((g, req.duration_hours))
    for h in [12, 6, 3]:
        if h < req.duration_hours: counter_candidates.append((req.granularity_min, h))

    best_counter = None
    max_counter_obj = -float('inf')

    for (cg, ch) in counter_candidates:
        temp_req = ContractRequest(req.requester_id, req.owner_id, req.features, cg, ch, req.purpose, req.proposed_eps)
        c_res = optimize_eps(temp_req, H_rem, Sx, Tij, Pp, lambdas)
        if c_res["feasible"] and c_res["obj"] > max_counter_obj:
            max_counter_obj = c_res["obj"]
            best_counter = (cg, ch, c_res)

    if best_counter:
        cg, ch, cr = best_counter
        scores.update({"counter_g": cg, "counter_h": ch, "counter_eps": cr["eps_star"]})
        return DecisionResult("counter", cr["eps_star"], 1e-6,
                              {"granularity_min": cg, "duration_hours": ch}, scores, reasons + ["COUNTER_FOUND"])

    return DecisionResult("reject", None, 1e-6, None, scores, reasons + ["NO_FEASIBLE_COUNTER"])

# =================================================================
# 4. X-CONTRACT (EXPLANATION LAYER)
# =================================================================

def explain_decision(req: ContractRequest, res: DecisionResult) -> Dict:
    """Generates human-centric narratives for the Household and Requester."""
    s = res.scores
    pu_summary = round(max(0.0, min(1.0, (s["objective"] + 1.5) / 3.0)), 2)

    if res.decision == "approve":
        h_text = f"Request approved. Budget ε*={res.eps_star} allocated from {s['H_rem']} remaining. Risk is low ({s['R(eps*)']})."
        r_text = f"Approved. ε*={res.eps_star}. Proceed with {req.granularity_min}m resolution code in Execution Sandbox."
    elif res.decision == "counter":
        p = res.counter_params
        h_text = f"Original request too risky for current budget. Counter-offered at {p['granularity_min']}m resolution."
        r_text = f"Counter-offer: ε*={res.eps_star} is available if you use {p['granularity_min']}m resolution and {p['duration_hours']}h window."
    else:
        h_text = f"Rejected. Request requires ε > {s['eps_min']}, but only {s['H_rem']} remains for this sensitivity level."
        r_text = "Rejected. Privacy constraints cannot be met. Try requesting less sensitive features or lower resolution."

    return {
        "summary": asdict(res),
        "pu_score": pu_summary,
        "narratives": {"household": h_text, "requester": r_text}
    }

# =================================================================
# 5. EXECUTION DEMO
# =================================================================

def run_simulation():
    history = {("aggregator_vpp", "home_001"): {"success_count": 15, "quality": 0.9, "purpose_match": 0.9}}
    lambdas = {"u": 1.0, "r": 1.2, "t": 0.4, "p": 0.5, "c": 0.1}

    # Test Case: High resolution request on low budget
    req = ContractRequest(
        requester_id="aggregator_vpp",
        owner_id="home_001",
        features=["load_curve", "appliances"],
        granularity_min=5,
        duration_hours=24,
        purpose="demand_response",
        proposed_eps=4.0
    )

    print("--- X-NegoBox Decision Log ---")
    for budget in [5.0, 1.5, 0.3]:
        result = apbnp_negotiate(req, budget, history, lambdas)
        explanation = explain_decision(req, result)
        print(f"\n[Budget H={budget}] Decision: {result.decision.upper()}")
        print(f"Household: {explanation['narratives']['household']}")
        print(f"Requester: {explanation['narratives']['requester']}")

if __name__ == "__main__":
    run_simulation()

--- X-NegoBox Decision Log ---

[Budget H=5.0] Decision: APPROVE
Household: Request approved. Budget ε*=2.25 allocated from 5.0 remaining. Risk is low (0.157).
Requester: Approved. ε*=2.25. Proceed with 5m resolution code in Execution Sandbox.

[Budget H=1.5] Decision: APPROVE
Household: Request approved. Budget ε*=1.5 allocated from 1.5 remaining. Risk is low (0.117).
Requester: Approved. ε*=1.5. Proceed with 5m resolution code in Execution Sandbox.

[Budget H=0.3] Decision: REJECT
Household: Rejected. Request requires ε > 1.37, but only 0.3 remains for this sensitivity level.
Requester: Rejected. Privacy constraints cannot be met. Try requesting less sensitive features or lower resolution.


In [ ]:
import math
import json
from dataclasses import dataclass, asdict, field
from typing import List, Dict, Tuple, Optional

# =================================================================
# 1. DATA STRUCTURES
# =================================================================

@dataclass
class ContractRequest:
    requester_id: str
    owner_id: str
    features: List[str]
    granularity_min: int
    duration_hours: int
    purpose: str
    proposed_eps: float

@dataclass
class DecisionResult:
    decision: str
    eps_star: Optional[float]
    delta: float
    counter_params: Optional[Dict] = None
    scores: Dict[str, float] = field(default_factory=dict)
    reason_codes: List[str] = field(default_factory=list)

# System Constants
ALPHA = {"location": 1.0, "appliances": 0.7, "load_curve": 0.4, "aggregate": 0.2}
P_SCORE = {"billing": 1.0, "demand_response": 0.7, "peer_trading": 0.6, "profiling": 0.1}

# =================================================================
# 2. DECISION LOGIC (APBNP)
# =================================================================

def calculate_utility(eps: float, granularity: int, duration: int) -> float:
    if eps <= 0: return 0.0
    eps_term = 1.0 - math.exp(-eps / 3.0)
    gran_term = min(60.0 / max(granularity, 1), 4.0) / 4.0
    dur_term = min(duration / 24.0, 1.0)
    return 0.55 * eps_term + 0.25 * gran_term + 0.20 * dur_term

def calculate_risk(eps: float, Sx: float, H_rem: float) -> float:
    budget_pressure = 1.0 - min(H_rem / 10.0, 1.0)
    return (eps / 10.0) * (0.6 + 0.4 * min(Sx / 2.0, 1.0)) * (0.7 + 0.3 * budget_pressure)

def optimize_eps(req: ContractRequest, H_rem: float, Sx: float, Tij: float, Pp: float, lambdas: Dict) -> Dict:
    """Core optimization loop finding the best privacy/utility balance."""
    best_eps, best_obj = None, -float('inf')
    eps_limit = min(10.0, H_rem)

    # Calculate minimum epsilon needed for the data to be 'useful'
    eps_min = 0.8 + (0.6 * min(Sx / 2.0, 1.0)) + (0.8 * (1.0 - Pp))

    if eps_limit < 0.25:
        return {"feasible": False, "eps_min": eps_min, "obj": -1.0, "eps_star": None}

    for eps in [round((i + 1) * 0.25, 2) for i in range(int(eps_limit / 0.25))]:
        U = calculate_utility(eps, req.granularity_min, req.duration_hours)
        R = calculate_risk(eps, Sx, H_rem)
        obj = (lambdas["u"] * U - lambdas["r"] * R + lambdas["t"] * Tij + lambdas["p"] * Pp)
        if obj > best_obj:
            best_obj, best_eps = obj, eps

    return {
        "eps_star": best_eps,
        "obj": best_obj,
        "feasible": (best_eps is not None and best_eps >= eps_min),
        "eps_min": eps_min
    }

def apbnp_negotiate(req: ContractRequest, H_rem: float, history: Dict, lambdas: Dict) -> DecisionResult:
    """Negotiates privacy budget and handles counter-offers."""
    Sx = sum(ALPHA.get(f, 0.5) for f in req.features)
    rec = history.get((req.requester_id, req.owner_id), {})
    Tij = (0.4 * min(rec.get("success_count", 0)/20, 1)) + (0.6 * rec.get("quality", 0.5))
    Pp = P_SCORE.get(req.purpose, 0.3)

    res = optimize_eps(req, H_rem, Sx, Tij, Pp, lambdas)

    # Initialize scores to prevent KeyErrors
    scores = {
        "Sx": Sx,
        "H_rem": H_rem,
        "eps_min": res["eps_min"],
        "obj": res["obj"],
        "U": calculate_utility(res["eps_star"], req.granularity_min, req.duration_hours) if res["eps_star"] else 0.0
    }

    if res["feasible"]:
        return DecisionResult("approve", res["eps_star"], 1e-6, None, scores)

    # Counter-offer logic (Try reducing detail from 5-min to 60-min)
    if req.granularity_min < 60:
        req_c = ContractRequest(req.requester_id, req.owner_id, req.features, 60, req.duration_hours, req.purpose, req.proposed_eps)
        res_c = optimize_eps(req_c, H_rem, Sx, Tij, Pp, lambdas)
        if res_c["feasible"]:
            scores.update({"counter_eps": res_c["eps_star"], "obj": res_c["obj"]})
            return DecisionResult("counter", res_c["eps_star"], 1e-6, {"granularity_min": 60}, scores)

    return DecisionResult("reject", None, 1e-6, None, scores)

# =================================================================
# 3. LAYMAN EXPLANATION LAYER (X-CONTRACT)
# =================================================================

def get_layman_explanation(req: ContractRequest, res: DecisionResult):
    """Translates technical math into 'What, Why, How, When' for a homeowner."""

    impact = "low" if res.scores["Sx"] < 0.4 else "moderate" if res.scores["Sx"] < 0.8 else "high"

    if res.decision == "approve":
        what = "✅ Request Approved"
        why = (f"This company asked for {impact} sensitivity info ({', '.join(req.features)}). "
               f"Since you have a healthy 'Privacy Wallet' balance, I've allowed it.")
        how = "I've added digital 'noise' to your data. They can see trends (like peak usage) but cannot see exactly what you were doing."
        when = "The data has been processed and shared securely."

    elif res.decision == "counter":
        what = "⚠️ Suggested Change"
        why = (f"They asked for very detailed data (every {req.granularity_min} minutes). "
               f"That's too much for your current privacy budget. I suggested giving them hourly data instead.")
        how = "By 'zooming out,' we keep your specific habits private while still being helpful to the grid."
        when = "I am waiting for them to accept the less-detailed version."

    else:
        what = "🚫 Request Blocked"
        why = (f"This request was for {impact} sensitivity data. You are currently low on your 'Privacy Wallet' units. "
               f"Sharing this now would be like giving away too many personal secrets.")
        how = "No data was shared. I protected your information by stopping the process."
        when = "The request was declined automatically."

    return {
        "What": what,
        "Why": why,
        "How it works": how,
        "Next Step": when,
        "Privacy Wallet": f"{round(res.scores['H_rem'], 2)} units left today"
    }

# =================================================================
# 4. EXECUTION
# =================================================================

def run_demo():
    history = {("aggregator_vpp", "home_001"): {"success_count": 18, "quality": 0.9}}
    lambdas = {"u": 1.0, "r": 1.2, "t": 0.4, "p": 0.5}

    # Request: High detail (5 min) load curve
    req = ContractRequest("aggregator_vpp", "home_001", ["load_curve"], 5, 24, "demand_response", 4.0)

    print("="*60)
    print("X-NEGOBOX: YOUR PERSONAL PRIVACY ASSISTANT")
    print("="*60)

    for budget in [5.0, 1.2, 0.2]:
        result = apbnp_negotiate(req, budget, history, lambdas)
        layman = get_layman_explanation(req, result)

        print(f"\n[SITUATION: Privacy Wallet Balance = {budget}]")
        print("-" * 45)
        for key, value in layman.items():
            print(f"{key:15}: {value}")
        print("-" * 45)

if __name__ == "__main__":
    run_demo()

X-NEGOBOX: YOUR PERSONAL PRIVACY ASSISTANT

[SITUATION: Privacy Wallet Balance = 5.0]
---------------------------------------------
What           : ✅ Request Approved
Why            : This company asked for moderate sensitivity info (load_curve). Since you have a healthy 'Privacy Wallet' balance, I've allowed it.
How it works   : I've added digital 'noise' to your data. They can see trends (like peak usage) but cannot see exactly what you were doing.
Next Step      : The data has been processed and shared securely.
Privacy Wallet : 5.0 units left today
---------------------------------------------

[SITUATION: Privacy Wallet Balance = 1.2]
---------------------------------------------
What           : 🚫 Request Blocked
Why            : This request was for moderate sensitivity data. You are currently low on your 'Privacy Wallet' units. Sharing this now would be like giving away too many personal secrets.
How it works   : No data was shared. I protected your information by stopping the

In [ ]:
from dataclasses import dataclass
from enum import Enum


# -----------------------------
# Decision types
# -----------------------------
class Decision(Enum):
    APPROVE = "approve"
    REJECT = "reject"
    COUNTER = "counter"


# -----------------------------
# Request and state definitions
# -----------------------------
@dataclass
class DataRequest:
    feature_type: str          # e.g., aggregate, load_curve, appliance
    resolution_minutes: int    # temporal granularity
    duration_days: int
    sensitivity_score: float   # S_x
    required_epsilon: float    # minimum epsilon needed


@dataclass
class PrivacyState:
    remaining_budget: float    # H_j^rem


# -----------------------------
# Core APBNP decision logic
# -----------------------------
def apbnp_decision(request: DataRequest, state: PrivacyState):
    """
    Simplified APBNP logic for experimentation.
    """
    if request.required_epsilon <= state.remaining_budget:
        decision = Decision.APPROVE
        epsilon_star = request.required_epsilon
    else:
        decision = Decision.REJECT
        epsilon_star = None

    return decision, epsilon_star


# -----------------------------
# X-Contract explanation layer
# -----------------------------
def explain_to_household(decision, request, state, epsilon_star):
    if decision == Decision.APPROVE:
        if state.remaining_budget - epsilon_star > 1.0:
            return (
                "Your data request has been approved.\n\n"
                "The information being shared is not highly sensitive and "
                "does not reveal detailed daily behaviour. You have enough "
                "privacy allowance remaining, so sharing this data is considered safe.\n\n"
                "The computation will run securely on your device, and only "
                "privacy-protected results will be shared. No raw data leaves your home.\n\n"
                "No action is required from you."
            )
        else:
            return (
                "Your data request has been approved, but your remaining privacy allowance is now low.\n\n"
                "This request is still safe, but it uses most of your available privacy capacity. "
                "Future requests may need to be more limited or less detailed.\n\n"
                "Your data remains protected, and nothing leaves your device without safeguards."
            )

    if decision == Decision.REJECT:
        return (
            "This data request has been rejected to protect your privacy.\n\n"
            "The request would require sharing information that is too detailed "
            "given your current privacy allowance. Approving it could reveal "
            "sensitive household patterns.\n\n"
            "No action is required from you. Your data remains fully protected."
        )


def explain_to_requester(decision, request, state, epsilon_star):
    if decision == Decision.APPROVE:
        return (
            f"Request approved.\n"
            f"Allocated privacy budget ε* = {epsilon_star:.2f}.\n"
            f"Proceed with {request.resolution_minutes}-minute resolution "
            f"for {request.duration_days} day(s) in the execution sandbox."
        )

    if decision == Decision.REJECT:
        return (
            "Request rejected.\n"
            f"Required privacy budget exceeds remaining allowance "
            f"({state.remaining_budget:.2f}).\n"
            "Consider requesting lower temporal resolution, shorter duration, "
            "or less sensitive features."
        )


# -----------------------------
# Experimental scenarios
# -----------------------------
def run_experiment():
    scenarios = [
        {
            "state": PrivacyState(remaining_budget=5.0),
            "request": DataRequest(
                feature_type="aggregate",
                resolution_minutes=5,
                duration_days=1,
                sensitivity_score=0.2,
                required_epsilon=2.25
            )
        },
        {
            "state": PrivacyState(remaining_budget=1.5),
            "request": DataRequest(
                feature_type="aggregate",
                resolution_minutes=5,
                duration_days=1,
                sensitivity_score=0.2,
                required_epsilon=1.5
            )
        },
        {
            "state": PrivacyState(remaining_budget=0.3),
            "request": DataRequest(
                feature_type="load_curve",
                resolution_minutes=1,
                duration_days=1,
                sensitivity_score=0.6,
                required_epsilon=1.37
            )
        }
    ]

    for i, s in enumerate(scenarios, start=1):
        decision, eps = apbnp_decision(s["request"], s["state"])

        print("\n" + "-" * 60)
        print(f"Scenario {i}")
        print(f"Remaining budget: {s['state'].remaining_budget}")

        print("\n[Household explanation]")
        print(explain_to_household(decision, s["request"], s["state"], eps))

        print("\n[Requester explanation]")
        print(explain_to_requester(decision, s["request"], s["state"], eps))


# -----------------------------
# Run experiment
# -----------------------------
if __name__ == "__main__":
    run_experiment()



------------------------------------------------------------
Scenario 1
Remaining budget: 5.0

[Household explanation]
Your data request has been approved.

The information being shared is not highly sensitive and does not reveal detailed daily behaviour. You have enough privacy allowance remaining, so sharing this data is considered safe.

The computation will run securely on your device, and only privacy-protected results will be shared. No raw data leaves your home.

No action is required from you.

[Requester explanation]
Request approved.
Allocated privacy budget ε* = 2.25.
Proceed with 5-minute resolution for 1 day(s) in the execution sandbox.

------------------------------------------------------------
Scenario 2
Remaining budget: 1.5

[Household explanation]
Your data request has been approved, but your remaining privacy allowance is now low.

This request is still safe, but it uses most of your available privacy capacity. Future requests may need to be more limited or less d